<a href="https://colab.research.google.com/github/williap2745/pyserpent/blob/Implementation-branch/Serpent_Cipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/williap2745/pyserpent


Cloning into 'pyserpent'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 31 (delta 6), reused 0 (delta 0), pack-reused 9 (from 1)
Receiving objects: 100% (31/31), 22.57 KiB | 608.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
%cd pyserpent


/content/pyserpent


In [3]:
!ls

README	serpent.py


In [4]:
import lib2to3.refactor
from pathlib import Path

# Path to the serpent.py file
serpent_file = Path('/content/pyserpent/serpent.py')

# Load the code
code = serpent_file.read_text()

# Use 2to3 refactoring tool
tool = lib2to3.refactor.RefactoringTool(lib2to3.refactor.get_fixers_from_package('lib2to3.fixes'))
fixed_tree = tool.refactor_string(code, str(serpent_file))
serpent_file.write_text(str(fixed_tree))

<ipython-input-4-f234be6949b6>:1: DeprecationWarning: lib2to3 package is deprecated and may not be able to parse Python 3.10+
  import lib2to3.refactor


68858

In [5]:
file_path = "/content/pyserpent/serpent.py"

# Read the current content of the file
with open(file_path, "r") as f:
    lines = f.readlines()

patched_lines = []

in_function = False
function_indent_level = 0

# Iterate over each line to find the function where key or block is used and adjust its indentation
for line in lines:
    stripped_line = line.strip()

    # Skip the __init__ function by checking for 'def __init__'
    if stripped_line.startswith("def __init__"):
        patched_lines.append(line)
        continue

    # Skip any function that doesn't contain 'key' or 'block' if you don't want the checks
    if stripped_line.startswith("def ") and "key" not in stripped_line and "block" not in stripped_line:
        patched_lines.append(line)
        continue

    # Detect function definitions and detect indentation level (for functions that use 'key')
    if stripped_line.startswith("def ") and "key" in stripped_line and "):" in stripped_line:
        in_function = True
        function_indent_level = len(line) - len(line.lstrip())  # Get the indentation level for the function
        patched_lines.append(line)  # Keep the function signature
        continue

    # Detect functions with 'block' and apply encoding for block
    if stripped_line.startswith("def ") and "block" in stripped_line and "):" in stripped_line:
        in_function = True
        function_indent_level = len(line) - len(line.lstrip())  # Get the indentation level for the function
        patched_lines.append(line)  # Keep the function signature
        continue

    if in_function:
        # Check for the indentation of each line within the function
        if stripped_line == "":  # For empty lines, keep them as they are
            patched_lines.append(line)
        else:
            # Add the encoding fix for key or block
            if "key" in stripped_line:
                patched_lines.append(" " * function_indent_level + "    if isinstance(key, str):\n")
                patched_lines.append(" " * function_indent_level + "        key = key.encode('utf-8')\n")
            elif "block" in stripped_line:
                patched_lines.append(" " * function_indent_level + "    if isinstance(block, str):\n")
                patched_lines.append(" " * function_indent_level + "        block = block.encode('utf-8')\n")

            in_function = False  # Stop after modifying the relevant function

        # Append the original line under the function
        patched_lines.append(line)

    else:
        patched_lines.append(line)

# Save patched file with fixed indentation
with open(file_path, "w") as f:
    f.writelines(patched_lines)

print("✅ serpent.py patched, skipping __init__ and handling both 'key' and 'block'.")


✅ serpent.py patched, skipping __init__ and handling both 'key' and 'block'.


In [6]:
import sys
sys.path.append('/content/pyserpent')  # Path to the cloned repo

import importlib.util

# Load the serpent.py file
spec = importlib.util.spec_from_file_location("serpent", "/content/pyserpent/serpent.py")
serpent = importlib.util.module_from_spec(spec)
spec.loader.exec_module(serpent)

In [7]:
#PKSC7 padding and unpadding
def pad(data):
    pad_len = 16 - (len(data) % 16)
    return data + bytes([pad_len] * pad_len)

def unpad(data):
    pad_len = data[-1]
    return data[:-pad_len]

In [8]:
# Import directly from the file
from serpent import Serpent

In [14]:
# Test key and plaintext
key = b"thisisasecretkey"      # 16 bytes
plaintext = b"I am finally done trouble shooting this. YAY!"  # 15 bytes, will need padding

#displaying original text
print("Plaintext:", plaintext)

# Pad the plaintext
padded = pad(plaintext)

# Encrypt
cipher = Serpent(key)
ciphertext = cipher.encrypt(padded)

print("Ciphertext:", ciphertext)

# Assuming your Serpent class has a decrypt method:
decrypted = cipher.decrypt(ciphertext)

# Remove padding
def unpad(data):
    padding_len = data[-1]
    return data[:-padding_len]

print("Decrypted:", unpad(decrypted))


Plaintext: b'I am finally done trouble shooting this. YAY!'
Ciphertext: b"'\xfe\x1f\xbb\xf9zMi\x8f2\xc7\xc0K;\x96\xee\xadxX\x89\xda3\xa8\x9eY}\xf7'\xd6L\xbb\xaf\x81\x13vH&(s\xdb\xa8\x90\xb3h{\x10p\xb3"
Decrypted: b'I am finally done trouble shooting this. YAY!'


In [10]:
# Assuming your Serpent class has a decrypt method:
decrypted = cipher.decrypt(ciphertext)

# Remove padding
def unpad(data):
    padding_len = data[-1]
    return data[:-padding_len]

print("Decrypted:", unpad(decrypted))


Decrypted: b'test message123'


In [16]:
!grep -rnw '.' -e 'def serpent_cbc_encrypt'


./serpent.py:2976:def serpent_cbc_encrypt(key, data, iv='\x00'*16):
